In [1]:
import numpy as np
import pandas as pd

from functools import reduce
from collections import Counter

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# em cima do dataset 'spam_ham.csv', seleciono a métrica de interesse e faço a otimização de hiperparâmteros
# a fim de encontrar o melhor modelo para o problema

In [2]:
# criando 'Bag of Words'

df = pd.read_csv('spam_ham.csv')

df.text = df.text.str.lower().str.replace(r'[^\w\s]+', ' ').str.split()

mensagens_juntas = reduce(lambda x, y: x + y, df.text)
palavras_comuns = Counter(mensagens_juntas).most_common(100)
palavras_comuns = [i[0] for i in palavras_comuns]

x = []
for mensagem in df.text:
    contagem = {p: 0 for p in palavras_comuns}
    
    for palavra in mensagem:
        if palavra in contagem:
            contagem[palavra] += 1
    
    x.append(contagem)
    
x = pd.DataFrame(x)
y = df.type.apply(lambda x: int(x == 'spam'))

# pré processamento
x = MinMaxScaler().fit_transform(x)

# split de treino e teste
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [3]:
# métrica de avaliação: fbeta
# caso positivo: mensagem spam
# caso negativo: mensagem não spam

# a métrica 'fbeta' é a média harmônica ponderada da 'precisão' e da 'revocação', estas que representam
# respectivamente, o número de verdadeiros positivos sobre o total de predições positivas(uma precisão 
# alta indica que ocorreram poucos falsos positivos) e o número de verdadeiros positivos sobre o total de 
# elementos positivos (reflete quantos dos casos positivos foram detectados)

# optei pela 'fbeta', posto que, do meu ponto de vista, a prioridade é minimizar o número de falsos positivos
# (dizer que uma mensagem não spam é spam), uma vez que a ocorrência dos mesmos poderia incorrer na perda de 
# informações importantes (por exemplo, no caso de uma conta de e-mail, uma mensagem importante, 
# incorretamente classificada como spam é direcionada à quarentena; o usuário não acessa esta seção de sua 
# conta e deixa de receber o conteúdo daquele e-mail), no entanto, o modelo não seria útil, caso trouxesse
# zero falsos positivos, mas sua revocação fosse muito baixa (por exemplo, o modelo acusa apenas um elemento 
# como positivo e acerta, resultando em uma precisão perfeita, mas a base de dados continha cem casos positivos, 
# resultando em uma revocação de 0.1), portanto, uma solução que julgo adequada é uma métrica que leve em 
# consideração estes dois fatores, a porporção de falsos positivos e a proporção de positivos detectados, 
# mas possa priorizar a não ocorrência de falsos positivos(por isso que o peso da precisão, no cálculo da média,
# é maior)

# ------------------------------------------------
# o motivo de se tirar a média harmônica, e não a média aritmética: 

# a primeira pune valores extremos, por exemplo, se fôssemos calcular o f1-score(média harmônica entre precisão
# e revocação, onde estas possuem o mesmo peso), para um f1 alto, é necessário que tanto a precisão quanto a 
# revocação sejam altas; tendo valores x = 1, y = 0.1, a média aritmética é 0.55, já a média harmônica é 0.18

In [4]:
# dicionário contendo algoritmos e seus repectivos hiperparâmetros; este será percorrido por um 'for'

model_params = {
                 'mnb':{'model': MultinomialNB(),
                       'params':{} },
                 
                 'dt':{'model': DecisionTreeClassifier(),
                       'params':{'max_depth':[4, 8, 12, 16], 
                                 'min_samples_split':[2, 4, 8, 12], 
                                 'min_samples_leaf':[2, 4, 6, 8, 12]} },
                        
                 'lr':{'model': LogisticRegression(), 
                      'params':{'solver':['lbfgs', 'liblinear'],
                                'class_weight':[None, 'balanced']} },
                        
                 'knn':{'model': KNeighborsClassifier(),
                       'params':{'n_neighbors':[2, 3, 5, 7, 9, 11, 13, 15, 17, 19],
                                 'weights':['uniform', 'distance']} }
                }

In [5]:
# GridSearchCV rodando com dados de treino apenas

my_scorer = make_scorer(fbeta_score, beta = 0.5)
scores = []
for m_name, m_prms in model_params.items():
    gscv =  GridSearchCV(m_prms['model'], m_prms['params'], cv = 5, scoring = my_scorer)
    gscv.fit(X_train, y_train)
    
    scores.append({
                     'model': m_name,
                     'best_score': gscv.best_score_,
                     'best_params': gscv.best_params_
                 })
    
for i in scores:
    print('{0} : {1}%\n{2}\n'.format(i['model'], round((i['best_score'] * 100), 2),i['best_params']))

mnb : 81.59%
{}

dt : 87.81%
{'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 4}

lr : 88.05%
{'class_weight': None, 'solver': 'liblinear'}

knn : 89.17%
{'n_neighbors': 9, 'weights': 'distance'}



In [ ]:
lr = LogisticRegression(class_weight = None, solver = 'liblinear')
knn = KNeighborsClassifier(n_neighbors = 9, weights = 'distance')

In [6]:
# dados de teste nos dois melhores modelos, com os melhores hiperparâmetros, de acordo com o grid search

knn = KNeighborsClassifier(n_neighbors = 9, weights = 'distance') 
knn.fit(X_train, y_train)
knn_score = fbeta_score(y_pred = knn.predict(X_test), y_true = y_test, beta = 0.5)

lr = LogisticRegression(class_weight = None, solver = 'liblinear')
lr.fit(X_train, y_train)
lr_score = fbeta_score(y_pred = lr.predict(X_test), y_true = y_test, beta = 0.5)

print('KNeighbors: {0}%\nLogisticRegression: {1}%'.format(round((knn_score * 100), 2), 
                                                          round((lr_score * 100), 2)))

KNeighbors: 89.13%
LogisticRegression: 86.59%


In [7]:
# opto pelo algortimo 'KNeighbors', com hiperparâmetros: n_neighbors = 9 e weights = 'distance',
# posto que este obteve o 'score' mais alto e se super-ajustou menos